
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img
    src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png"
    alt="Databricks Learning"
  >
</div>


# Status Pipeline

In this DLT pipeline, the **status** pipeline is implemented within a notebook. DLT supports using `.py`, `.sql`, or notebook files as pipeline sources.

**NOTE:** In an notebook you must use either Python or SQL. 

The final pipeline performs the following tasks:

1. Creates the **status_bronze_demo5** streaming table by ingesting raw JSON files from `/Volumes/dbacademy/ops/your-lab-user/status/`.

2. Creates the **status_silver_demo5** streaming table from the **status_bronze_demo5** table.

3. Creates the materialized view **full_order_status_gold_demo5** to capture each order's status by joining the following tables:
   - **status_silver_demo5**
   - **orders_silver_demo5**

4. Creates the following materialized views:
   - **cancelled_orders_gold_demo5** – Displays all cancelled orders and how many days passed before cancellation.
   - **delivered_orders_gold_demo5** – Displays all delivered orders and how many days it took to deliver each order.
![Pipeline](../../Includes/images/demo6_pipeline_image.png)

## A. JSON -> Bronze
The code below ingests JSON files located in your `/Volumes/dbacademy/ops/your-lab-user/status/` volume, using the `source` DLT configuration parameter to point to the base path `/Volumes/dbacademy/ops/your-lab-user/`.

In [0]:
CREATE OR REFRESH STREAMING TABLE 1_bronze_db.status_bronze_demo6
  COMMENT "Ingest raw JSON order status files from cloud storage"
  TBLPROPERTIES (
    "quality" = "bronze",
    "pipelines.reset.allowed" = false     -- prevent full table refreshes on the bronze table
  )
AS 
SELECT 
  *,
  current_timestamp() processing_time, 
  _metadata.file_name AS source_file
FROM STREAM read_files(
  "${source}/status", 
  format => "json");

## B. Bronze -> Silver
The code below performs a simple transformation on the date field and selects only the necessary columns for the silver streaming table **status_silver_demo5**.  

We're also adding a comment and table properties to document the table for production use, along with DLT expectations to enforce data quality on the streaming table.

In [0]:
CREATE OR REFRESH STREAMING TABLE 2_silver_db.status_silver_demo6
  (
    -- Drop rows if order_status_timestamp is not valid
    CONSTRAINT valid_timestamp EXPECT (order_status_timestamp > "2021-12-25") ON VIOLATION DROP ROW,
    -- Warn if order_status is not in the following
    CONSTRAINT valid_order_status EXPECT (order_status IN ('on the way','canceled','return canceled','delivered','return processed','placed','preparing'))
  )
  COMMENT "Order with each status and timestamp"
  TBLPROPERTIES ("quality" = "silver")
AS 
SELECT
  order_id,
  order_status,
  timestamp(status_timestamp) AS order_status_timestamp
FROM STREAM 1_bronze_db.status_bronze_demo6;

## C. Use a Materialized View to Join Two Tables
One way to join two streaming tables in DLT is by creating a materialized view that performs the join.  This approach takes all rows from each streaming table and executes a full inner join operation.

**NOTES:**

- **Materialized views include built-in optimizations where applicable:**
  - [Incremental refresh for materialized views](https://docs.databricks.com/aws/en/optimizations/incremental-refresh)
  - [Delta Live Tables Announces New Capabilities and Performance Optimizations](https://www.databricks.com/blog/2022/06/29/delta-live-tables-announces-new-capabilities-and-performance-optimizations.html)
  - [Cost-effective, incremental ETL with serverless compute for Delta Live Tables pipelines](https://www.databricks.com/blog/cost-effective-incremental-etl-serverless-compute-delta-live-tables-pipelines)

- **Stateful joins (Stream to Stream):** For stateful joins in DLT (i.e., joining incrementally as data is ingested), refer to the [Optimize stateful processing in DLT with watermarks](https://docs.databricks.com/aws/en/dlt/stateful-processing) documentation. **Stateful joins are an advanced topic and outside the scope of this course.**

In [0]:
CREATE OR REFRESH MATERIALIZED VIEW 3_gold_db.full_order_info_gold_demo6
  COMMENT "Joining the orders and order status silver tables to view all orders with each individual status per order"
  TBLPROPERTIES ("quality" = "gold")
AS 
SELECT
  orders.order_id,
  orders.order_timestamp,
  status.order_status,
  status.order_status_timestamp
-- Notice that the STREAM keyword was not used when referencing the streaming tables
FROM 2_silver_db.status_silver_demo6 status    
  INNER JOIN 2_silver_db.orders_silver_demo6 orders 
  ON orders.order_id = status.order_id;

## D. Create Materialized Views for Cancelled and Delivered Orders

The code below will create two tables using the joined data from above:

- **3_gold_db.cancelled_orders_gold_demo5**
    - A materialized view containing all **cancelled** orders
    - number of days it took to cancel each order.

- **3_gold_db.delivered_orders_gold_demo5**
    - A materialized view containing all **delivered** orders
    - number of days it took to deliver each order.

    [datediff function](https://docs.databricks.com/aws/en/sql/language-manual/functions/datediff)

In [0]:
-- CANCELLED ORDERS MV
CREATE OR REFRESH MATERIALIZED VIEW 3_gold_db.cancelled_orders_gold_demo6
  COMMENT "All cancelled orders"
  TBLPROPERTIES ("quality" = "gold")
AS 
SELECT
  order_id,
  order_timestamp,
  order_status,
  order_status_timestamp,
  datediff(DAY,order_timestamp, order_status_timestamp) AS days_to_cancel -- calculate days to cancel
FROM 3_gold_db.full_order_info_gold_demo6
WHERE order_status = 'canceled';




-- DELIVERED ORDERS MV
CREATE OR REFRESH MATERIALIZED VIEW 3_gold_db.delivered_orders_gold_demo6
  COMMENT "All delivered orders"
  TBLPROPERTIES ("quality" = "gold")
AS 
SELECT
  order_id,
  order_timestamp,
  order_status,
  order_status_timestamp,
  datediff(DAY,order_timestamp, order_status_timestamp) AS days_to_delivery -- calculate days to deliver
FROM 3_gold_db.full_order_info_gold_demo6
WHERE order_status = 'delivered';

## E. Create the Production Pipeline
Follow the steps below to modify the pipeline settings and run the production pipeline.

1. Complete the following steps to modify your pipeline configuration for production:

   a. Select the **Settings** icon ![Pipeline Settings](../../Includes/images/pipeline_settings_icon.png) in the left navigation pane.

   b. In the **Pipeline settings** section, you can modify the **Pipeline name** and **Run as** settings.

      - Click the pencil icon ![pencil_settings_icon.png](../../Includes/images/pencil_settings_icon.png) next to **Run as**.

      - You can optionally change the executor of the pipeline to a service principal.  
        A service principal is an identity you create in Databricks for use with automated tools, jobs, and applications.  

        - For more information, see the [What is a service principal?](https://docs.databricks.com/aws/en/admin/users-groups/service-principals#what-is-a-service-principal) documentation.

      - For this course select **Cancel** and leave **Run as** set to your username.

   c. In the **Code assets** section, confirm that:

      - **Root folder** points to this DLT project (**5 - Deploying a DLT Pipeline to Production**).

      - **Source code** references the **orders** and **status** folders within this project.

   d. In the **Default location for data assets** section, confirm the following:

      - **Default catalog** is your **labuser** catalog.

      - **Default schema** is the **default** schema.

   e. In the **Compute** section, confirm that **Serverless** compute is selected.

   f. In the **Configuration** section, ensure that the `source` key is set to your data source volume path:  
      `/Volumes/dbacademy/ops/your-labuser-name`

   g. In the **Tags** section you can add takes to help determine usage per department/chargeback.. We will leave them as is. 

   h. In the **Advanced settings** section:

      - Expand **Advanced settings**.

      - Click **Edit advanced settings**.

      - In **Pipeline mode**, ensure **Triggered** is selected so the pipeline runs on a schedule.  
        - Alternatively, you can choose **Continuous** mode to keep the pipeline running at all times.  
        - For more details, see [Triggered vs. continuous pipeline mode](https://docs.databricks.com/aws/en/dlt/pipeline-mode).

      - In **PIpeline user mode** select **Production**.

      - For **Channel**, you can leave it as **Preview** for training purposes:
        - **Current** – Uses the latest stable Databricks Runtime version, recommended for production.
        - **Preview** – Uses a more recent, potentially less stable Runtime version, ideal for testing upcoming features.
        - View the [DLT release notes and the release upgrade process](https://docs.databricks.com/aws/en/release-notes/dlt/) documentation for more information.

      - In the **Event logs** section:
        - Select **Publish event log to metastore**.
        - Set **Event log name** to `event_log_demo_5`.
        - Set **Event log catalog** to your **labuser** catalog.
        - Set **Event log schema** to the **default** schema.

        **NOTE:** If the event log is not saved to the correct location, the event log exploration steps will not work properly in the main notebook.

   i. Click **Save** to save your DLT pipeline settings.

2. Once your pipeline is production-ready, you'll want to schedule it to run either on a time interval or continuously.

   For this demonstration, we’ll:
   - Schedule the pipeline to run every day at 8:00 PM.
   - Optionally configure notifications to alert you upon job **Start**, **Success**, and **Failure**.  
     *(If you don’t want email notifications, you can skip this step.)*

   Complete the following steps to schedule the pipeline:

   a. Select the **Schedule** button.

   b. For the job name, leave it as **5 - Deploying a DLT Pipeline to Production Project - labuser-name**.

   c. Below **Job name**, select **Advanced**.

   d. In the **Schedule** section, configure the following:
   - Set the **Day**.
   - Set the time to **20:00** (8:00 PM).
   - Leave the **Timezone** as default.
   - Select **More options**, and under **Notifications**, add your email to receive alerts for:
     - **Start**
     - **Success**
     - **Failure**

   e. Click **Create** to save and schedule the job.

  **NOTE:** You could also set the pipeline to run a few minutes after your current time to see it start through the scheduler.

3. After completing the pipeline settings and scheduling the pipeline, let's manually trigger the pipeline by selecting the **Run pipeline** button.

    While the pipeline is running, you can explore what the final pipeline will look like using the image below:

    ![DLT Pipeline Demo 6](../../Includes/images/demo6_pipeline_image.png) 

**NOTE:** Currently we have one JSON file in both **status** and **orders**.

4. After the pipeline has completed it's first run, complete the following:

   a. Examine the **Pipeline graph** and confirm:
      - 174 rows were read into the **orders_bronze** and **orders_silver** streaming tables
      - 536 rows were read into the **status_bronze** and **status_silver** streaming tables
      - 536 rows are in the **full_order_info_gold** materialized view
      - 7 rows are in the **orders_by_date_gold** materialized view
      - 8 rows are in the **cancelled_orders_gold** materialized view
      - 94 rows are in the **delivered_orders_gold** materialized view

   b. Go back to the main notebook **5 - Deploying a DLT Pipeline to Production**

   c. Complete the steps in step **D. Land Data Data to Your Data Source Volume**.

5. After you have landed **4** new files into the data source volume, run the pipeline to process the newly landed JSON files.

   Notice the following:

   a. The **status** bronze to silver flow ingests 410 new rows.

   b. The **orders** bronze to silver flow ingests 98 new rows.

   c. The **full_order_info_gold** materialized view join contains a total of 946 rows (the previous 536 rows + the new 410 rows).

   d. The **cancelled_orders_gold** materialized view contains 21 rows.

   e. The **delivered_orders_gold** materialized view contains 176 rows.

   f. The **orders_by_date_gold** materialized view contains 11 rows.

6. In the window at the bottom, select the **Expectations** link for the **status_silver_demo5** table. It should contain the value **2**. Notice that in this run, 7.6% (31 rows) for the **valid_order_status** expectation returned a warning.

This is something we would want to investigate and address in future stages of the pipeline.

7. Go back to the main notebook **5 - Deploying a DLT Pipeline to Production** and complete the steps in step **E. Monitor Your Pipeline with the Event Log**.

&copy; 2026 Databricks, Inc. All rights reserved. Apache, Apache Spark, Spark, the Spark Logo, Apache Iceberg, Iceberg, and the Apache Iceberg logo are trademarks of the <a href="https://www.apache.org/" target="_blank">Apache Software Foundation</a>.<br/><br/><a href="https://databricks.com/privacy-policy" target="_blank">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use" target="_blank">Terms of Use</a> | <a href="https://help.databricks.com/" target="_blank">Support</a>